In [1]:
import rich
import json
from itertools import count
import pandas as pd

from langchain.docstore.base import Docstore
from langchain.retrievers import ElasticSearchBM25Retriever

from guidance import Program
from guidance.llms import OpenAI
from datasets import load_dataset
from elasticsearch import Elasticsearch

/opt/homebrew/Caskroom/mambaforge/base/envs/zoning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from ipython_secrets import get_secret

OPENAI_API_KEY = get_secret("OPENAI_API_KEY")

In [3]:
data = load_dataset("xyzNLP/nza-ct-zoning-codes-text")
data = pd.concat((data["train"].to_pandas().drop(columns=["embeddings"]), data["test"].to_pandas().drop(columns=["embeddings"])))
data

Found cached dataset parquet (/Users/maxdumas/.cache/huggingface/datasets/xyzNLP___parquet/xyzNLP--nza-ct-zoning-codes-text-579051b4b18688cb/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00, 148.23it/s]


,Town,Page,Text
0,vernon,1,
1,vernon,2,"ZONING REGULATIONS\nTown of Vernon, Connecticu..."
2,vernon,3,SECTION 1 - GENERAL\n1.1\nThe following regula...
3,vernon,4,1.3\nThe boundaries of these zones are shown o...
4,vernon,5,1.3A\nProposed changes to the Zoning Map shall...
...,...,...,...
9812,chester,110,"TOWN OF CHESTER\nZONING REGULATIONS\nAPRIL 1, ..."
9813,chester,111,"TOWN OF CHESTER\nZONING REGULATIONS\nAPRIL 1, ..."
9814,chester,112,"TOWN OF CHESTER\nZONING REGULATIONS\nAPRIL 1, ..."
9815,chester,113,"TOWN OF CHESTER\nZONING REGULATIONS\nAPRIL 1, ..."


In [4]:
data.query("Town == 'ashford' & Page == 24").Text.values[0]

"Ashford, Connecticut - Zoning Regulations\nArticle 4B Residential/Agricultural Zone (RA)\nArticle 4B, Section 1\nPurpose\nA.\nThe purpose of the Residential-Agricultural Zone is to allow residential and complementary non-residential\ndevelopment consistent with the rural character of the town that is reflected in its forests, farm fields, historic\nbuildings, and unique landscapes.\nB. The further purpose of the Residential-Agricultural Zone is to promote the economic viability and operational\nsustainability of agricultural business in the town of Ashford as outlined in the Ashford Plan of Conservation and\nDevelopment. Specifically, these regulations are intended to:\n1. address food and fiber needs;\n2. enhance environmental quality and the natural resource base upon which the agricultural economy depends;\n3.\nmake the most efficient use of nonrenewable resources and on-farm resources and integrate, where appropriate,\nnatural biological cycles and controls;\n4. sustain the econom

In [5]:
class ElasticSearchDocStore(Docstore):
    def __init__(self, index_name: str, texts: list[str], reset_index=True):
        if reset_index:
            es = Elasticsearch("http://localhost:9200")  # default client
            es.indices.delete(index=index_name, ignore=[400, 404])
        
        self.retriever = ElasticSearchBM25Retriever.create(
            elasticsearch_url="http://localhost:9200",
            index_name=index_name,
            k1=2.0,
            b=0.75,
        )
        self.retriever.add_texts(texts=texts)

    def search(self, query):
        return self.retriever.get_relevant_documents(query)

In [11]:
template = """
{{#system~}}
You are ZoningGPT, a powerful assistant who is an expert in reviewing portions of Zoning documents and answering questions about them.

Pay close attention to your answers in the past and be sure to match their schema. You will be evaluated on the quality and correctness of your answers.

You must answer by using one of the functions at your disposal, or by returning a JSON formatted final answer. If you cannot answer the question, return the string "FAILURE".

You may use the functions as many times as you like.

{{>tool_def functions=functions}}
{{~/system}}

{{#block hidden=True}}
    {{#user~}}
    What is the minimum lot size, in square feet, for a single family home in the first listed residential district of the town of Ansonia, Connecticut?
    {{~/user}}

    {{#assistant~}}
    ```typescript
    functions.search_documents({
    "town_name": "ansonia",
    "query": "zoning district list"
    })```
    {{~/assistant}}

    {{#user~}}
    ARTICLE II - ESTALISHMENT OF DISTRICTS AND ZONING\nADMINISTRATION\nSection 205 Zoning Districts Established\nFor the purposes of these regulations, the City of Ansonia is hereby divided into the following\nclasses of districts:\nSection 210 Zoning Map\nThe boundaries of the districts specified in Section 205 are hereby established as shown on a\nmap entitled "Official Zoning Map, City of Ansonia", dated February 1990, including any\nspecial maps and boundary descriptions supplementary thereto, adopted and declared to be a part\nof this Ordinance. In addition, the boundaries of the Flood Plain District are as delineated in\nSection 220 - Flood Plain District.\nSection 215 - Determination of District Boundaries\nIn determining the boundaries of districts shown on the Official Zoning Map, the following rules\nshall apply\n215.1 Unless otherwise shown, the district boundaries shall be construed to coincide with\nthe center lines of streets, alleys, parkways, waterways and main tracks of railroads. -\n215.2\nWhere such boundaries are indicated as approximate following the property lines of\nparks or other publicly owned lands, such lines shall be construed to be such\n17\nCELL (1, 1): \nDistrict\nCELL (1, 2): \nMap Code\nCELL (2, 1): \nAAA Residence District\nCELL (2, 2): \nAAA\nCELL (3, 1): \nAA Residence District\nCELL (3, 2): \nAA\nCELL (4, 1): \nA Residence District\nCELL (4, 2): \nA\nCELL (5, 1): \nB Residence District\nCELL (5, 2): \nB\nCELL (6, 1): \nGA Multi-Family Residence District\nCELL (6, 2): \nGA\nCELL (7, 1): \nMM Multi-Family Residence District\nCELL (7, 2): \nMM\nCELL (8, 1): \nBB Multi-Family Residence District\nCELL (8, 2): \nBB\nCELL (9, 1): \nRR Multi-Family Residence-Retail District\nCELL (9, 2): \nRR\nCELL (10, 1): \nNR Neighborhood-Retail District\nCELL (10, 2): \nNR\nCELL (11, 1): \nC Central Commercial District\nCELL (11, 2): \nC\nCELL (12, 1): \nSC Special Commercial District\nCELL (12, 2): \nSC\nCELL (13, 1): \nLI Light Industrial District\nCELL (13, 2): \nLI\nCELL (14, 1): \nHI Heavy Industrial District\nCELL (14, 2): \nHI\nCELL (15, 1): \nFlood Plain District\nCELL (15, 2): \nCELL (16, 1): \nCP Commerce Park District\nCELL (16, 2): \nCP\nCELL (17, 1): \nCCZ City Center Overlay Zone\nCELL (17, 2): \nCCOZ\n
    {{~/user}}

    {{#assistant~}}
    ```typescript
    functions.search_documents({
    "town_name": "ansonia",
    "query": "AAA Residence District, minimum lot size, single family home"
    })```
    {{~/assistant}}

    {{#user~}}
    SCHEDULE D - MAXIMUM DWELLING DENSITY\nEffective: April 25, 2003.\n52\nCELL (1, 1): \nZoning District\nCELL (1, 2): \nMinimum Lot Size (In\nSquare Feet)\nCELL (1, 3): \nMax. Residential Density (Dwelling Units\nPer Acre)\nCELL (2, 1): \nAAA\nCELL (2, 2): \n60,000\nCELL (2, 3): \n0.59\nCELL (3, 1): \nAA\nCELL (3, 2): \n30,000\nCELL (3, 3): \n1.00\nCELL (4, 1): \nA\nCELL (4, 2): \n12,500\nCELL (4, 3): \n2.40\nCELL (5, 1): \nB\nCELL (5, 2): \n7,500\nCELL (5, 3): \n4.00\n
    {{~/user}}

    {{#assistant~}}
    {
        "rationale": "The table indicates that the minimum lot size for a single family home in the AAA Residence District is 60,000 square feet. So the answer is 60,000 square feet.",
        "answer": "60,000 square feet"
    }
    {{~/assistant}}

    {{#user~}}
    Which districts permit accessory dwelling units (ADUs) in the town of Ashford, CT?
    {{~/user}}

    {{#assistant~}}
    ```typescript
    functions.search_documents({
    "town_name": "ashford",
    "query": "zoning district list"
    })```
    {{~/assistant}}

    {{#user~}}
    Ashford, Connecticut - Zoning Regulations\nArticle 4\nZoning Districts\nArticle 4A, Section 1 Establishment of Zoning Districts\nFor the purposes of these regulations, the town of Ashford is hereby divided into the following zones:\n1. Residential - Agricultural (RA)\n2. General Commercial (C)\nArticle 4A, Section 2 Zoning Map\nA. The boundaries of Districts are established as shown on the "Official Zoning Map."\n1.\nThe Official Zoning Map shall be at a scale of 1" = 1000\' and identified by the signature of the Chairman of the\nCommission, and shall bear the date of the most recent zoning map amendment.\n2.\nWhen, in accordance with the provisions of these regulations, changes are made in district boundaries or other\nmatter portrayed on the Official Zoning Map, such changes shall be made on the Official Zoning Map together\nwith an entry on the Official Zoning Map as follows: "As amended to (date)." (Such date to be that of the most\nrecent amendment.) The Official Zoning Map shall be filed in the office of the Town Clerk and an updated copy\nshall be displayed in Ashford Town Land Use Office.\nB. Where the boundary between districts is shown following a street, watercourse, railroad or utility right of way, the\nboundary shall be the centerline thereof, unless otherwise indicated. Where the boundary is shown approximately in\nthe location of Boundary Lines or Lot Lines, then the Boundary Line or Lot Line shall be the district boundary line.\nWhere the boundary does not follow a street, watercourse, railroad or utility right of way, Boundary Line or Lot Line,\nthe district boundary line shall be determined using the scale on the Official Zoning Map.\nC. Where a District boundary divides a lot in one ownership into a residential and a nonresidential District or into two\nnonresidential Districts, the area and frontage requirements for that lot shall comply with those that are more\nrestrictive as set forth for such Districts. All other building requirements shall correspond with those of the particular\nDistrict in which a use, structure or building is established or constructed.\nD. Where the location of a district boundary line is uncertain, the Commission shall interpret the district boundary line.\nOFFICIAL DRAFT - EFFECTIVE 11/1/2014\n21\n
    {{~/user}}

    {{#assistant~}}
    ```typescript
    functions.search_documents({
    "town_name": "ansonia",
    "query": "Residential - Agricultural, RA, permitted uses"
    })```
    {{~/assistant}}

    {{#user~}}
    Ashford, Connecticut - Zoning Regulations\nArticle 4B Residential/Agricultural Zone (RA)\nArticle 4B, Section 1\nPurpose\nA.\nThe purpose of the Residential-Agricultural Zone is to allow residential and complementary non-residential\ndevelopment consistent with the rural character of the town that is reflected in its forests, farm fields, historic\nbuildings, and unique landscapes.\nB. The further purpose of the Residential-Agricultural Zone is to promote the economic viability and operational\nsustainability of agricultural business in the town of Ashford as outlined in the Ashford Plan of Conservation and\nDevelopment. Specifically, these regulations are intended to:\n1. address food and fiber needs;\n2. enhance environmental quality and the natural resource base upon which the agricultural economy depends;\n3.\nmake the most efficient use of nonrenewable resources and on-farm resources and integrate, where appropriate,\nnatural biological cycles and controls;\n4. sustain the economic viability of farm operations, and;\n5. maintain an agricultural friendly community.\nArticle 4B, Section 2 Permitted Uses\nThe following uses are permitted in the Residential-Agricultural Zone in accordance with the requirements found in\nArticle 5:\nArticle 4B, Section 3 Special Permit Uses\nA special permit may be issued for the following uses in the Residential - Agricultural Zone pursuant to Article 5C of\nthese regulations:\n1. Educational (including boarding of students), religious or philanthropic buildings, structures or uses, excluding\nhospitals and correctional institutions\n2. Rural Businesses, in accordance with Article 4B, Section 6 of these Regulations\nOFFICIAL DRAFT - EFFECTIVE 11/1/2014\n23\nCELL (1, 1): \n1. Single family detached dwellings\nCELL (1, 2): \n1. Single family detached dwellings\nCELL (2, 1): \n2. Two family dwelling\nCELL (2, 2): \n2. Two family dwelling\nCELL (3, 1): \n3.\nCELL (3, 2): \nIn-Law Apartments, as accessory to a single family dwelling\nCELL (4, 1): \n4. Agriculture, exclusive of Farmer's Markets and Farm Stores\nCELL (4, 2): \n4. Agriculture, exclusive of Farmer's Markets and Farm Stores\nCELL (5, 1): \n5. Farm Stands - 200 sq. ft. or less in compliance with Article 4B, Section 9\nCELL (5, 2): \n5. Farm Stands - 200 sq. ft. or less in compliance with Article 4B, Section 9\nCELL (6, 1): \n6.\nCELL (6, 2): \nHome Occupations in compliance with Article 4B, Section 8, including without limitation Family Day Care\nHomes\nCELL (7, 1): \n7. Community Residence, provided there shall be a minimum separating distance of 1,000 feet between the\nCELL (7, 2): \n7. Community Residence, provided there shall be a minimum separating distance of 1,000 feet between the\nCommunity Residence and any such other Community Residence.\nCELL (8, 1): \n8.\nCELL (8, 2): \nOutdoor Wood Burning Furnace in accordance with Connecticut Law\nCELL (9, 1): \n9.\nCELL (9, 2): \nAmateur Radio Towers and Antennas, provided the Height of Tower shall not exceed 200 feet\nCELL (10, 1): \nCELL (10, 2): \n10. Accessory uses customarily incidental to and associated with the above permitted uses\n
    {{~/user}}
    
    {{#assistant~}}
    ```typescript
    functions.search_documents({
    "town_name": "ansonia",
    "query": "General Commercial, C, permitted uses"
    })```
    {{~/assistant}}

    {{#user~}}
    Ashford, Connecticut - Zoning Regulations\nArticle 4C General Commercial Zone (GC)\nArticle 4C, Section 1\nPurpose\nA. The purpose of the General Commercial Zone, consistent with the Town's Plan of Conservation and Development, is\nto provide for meaningful and realistic commercial utilization of appropriate portions of the Town for a\ncomplimentary and integrated mixture of employment, shopping, entertainment and civic uses while preserving the\nTown's rural character.\nB. The further purpose of the General-Commercial Zone to promote the economic viability and operational\nsustainability of agricultural business in the town of Ashford. as outlined in the Ashford Plan of Conservation and\nDevelopment,.. Specifically, these regulations are intended to:\n1. address food and fiber needs;\n2.\nenhance environmental quality and the natural resource base upon which the agricultural economy depends;\n3.\nmake the most efficient use of nonrenewable resources and on-farm resources and integrate, where appropriate,\nnatural biological cycles and controls;\n4. sustain the economic viability of farm operations, and;\n5. maintain an agricultural friendly community.\nArticle 4C, Section 2 Permitted Uses\nThe following uses, containing not more than five thousand (5,000) square feet of floor area, are permitted in the General\nCommercial Zone in accordance with the site plan requirements found in Article 5:\n1. Retail stores\n2. Agriculture\n3. Food and beverage stores for the sale of groceries, fruit and meat; baked goods; dairy products - not including\nthe sale of liquor.\n4. Restaurants\n5. Farm Stands and Farm Stores in accordance with Article 4C, Section 5\n6. Town Sponsored Farmer's Markets\n7.\nPersonal Service Establishments\n8. Banking and Financial Institutions\n9. Repair shops (exclusive of motor vehicle service and repair stations)\n10. Fitness, Dance, or Sport training facilities.\n11. Business and professional offices\n12. Public buildings (without outdoor storage)\n13. Day Care Center\n14. Accessory uses customarily incidental to the above permitted uses.\nArticle 4C, Section 3\nSpecial Permit Uses\nOFFICIAL DRAFT - EFFECTIVE 11/1/2014\n37\n
    {{~/user}}

    {{#assistant~}}
    {
        "rationale": "The permitted uses for the Residential - Agricultural (RA) zone afford the creation of "in-law apartments", which are accesory dwelling units. The General Commercial (C) zone does not mention anything like this in its permitted uses, so we assume it is prohibited.",
        "answer": "Residential Agricultural: Permitted\nGeneral Commercial: Prohibited"
    }
    {{~/assistant}}

{{/block}}


{{#user~}}
    {{question}}
{{~/user}}

{{#geneach 'conversation' stop=False}}
    {{#assistant~}}
        {{gen 'this.response' stop='\\n' function_call='auto'}}
    {{~/assistant}}

    {{#if callable(this.response)}}
        {{~#function name=this.response.__name__~}}
            {{this.response()}}
        {{~/function~}}
    {{else}}
        {{break}}
    {{/if}}
{{/geneach}}
"""

In [12]:
def search_documents(town_name: str, query: str):
    rich.print(town_name, query)
    docstore = ElasticSearchDocStore(town_name, data.query("Town == @town_name")["Text"].to_list())
    return docstore.search(query)[0].page_content

tools_dict = {
    "search_documents": search_documents
}

functions=[
    {
        "name": "search_documents",
        "description": "Searches the document store for the contents of a document matching the query. Use this tool to learn more about a topic.",
        "parameters": {
            "type": "object",
            "properties": {
                "town_name": {
                    "type": "string",
                    "description": "The name of the town to search in. Must be in all lowercase and have spaces converted to dashes, e.g. 'Beacon Falls' becomes 'beacon-falls'.",
                },
                "query": {
                    "type": "string",
                    "description": "A keyword search query, e.g. 'AAA Residence District, minimum lot size, single family home'",
                },
            },
            "required": ["town_name", "query"],
        }
    }
]

In [16]:
p = Program(template, llm=OpenAI("gpt-4-0613", api_key=OPENAI_API_KEY), caching=False, silent=False, log=True)
t = p(
    question='Which districts permit single family homes in the town of Ashford, CT?',
    functions=functions,
    **tools_dict
)

Stop program system You are ZoningGPT, a powerful assistant who is an expert in reviewing portions of Zoning documents and answering questions about them.

Pay close attention to your answers in the past and be sure to match their schema. You will be evaluated on the quality and correctness of your answers.

You must answer by using one of the functions at your disposal, or by returning a JSON formatted final answer. If you cannot answer the question, return the string "FAILURE".

You may use the functions as many times as you like.


# Tools

<span style='background-color: rgba(0, 138.56128016, 250.76166089, 0.25); display: inline;' title='{{#if len(functions) > 0~}}
## functions

namespace functions {

{{#each functions item_name="function"~}}
// {{function.description}}
type {{function.name}} = (_: {
{{~#each function.parameters.properties}}
{{#if contains(this, "description")}}// {{this.description}}
{{/if~}}
{{@key}}{{#unless contains(function.parameters.required, @key)}}?{{/unless}}: {{#if contains(this, "enum")}}{{#each this.enum}}"{{this}}"{{#unless @last}} | {{/unless}}{{/each}}{{else}}{{this.type}}{{/if}}{{#unless @last}},{{/unless}}
{{~/each}}
}) => any;

{{/each~}}
} // namespace functions
{{~/if~}}'>## functions

namespace functions {

<span style='opacity: 1.0; display: inline; background-color: rgba(165, 165, 165, 0.1);' title='{{#each functions item_name="function"~}}
// {{function.description}}
type {{function.name}} = (_: {
{{~#each function.parameters.properties}}
{{#if contains(this, "description")}}// {{this.description}}
{{/if~}}
{{@key}}{{#unless contains(function.parameters.required, @key)}}?{{/unless}}: {{#if contains(this, "enum")}}{{#each this.enum}}"{{this}}"{{#unless @last}} | {{/unless}}{{/each}}{{else}}{{this.type}}{{/if}}{{#unless @last}},{{/unless}}
{{~/each}}
}) => any;

{{/each~}}
'>// Searches the document store for the contents of a document matching the query. Use this tool to learn more about a topic. 
type search_documents = (_: { 
 // The name of the town to search in. Must be in all lowercase and have spaces converted to dashes, e.g. 'Beacon Falls' becomes 'beacon-falls'. 
 town_name : string , 
 // A keyword search query, e.g. 'AAA Residence District, minimum lot size, single family home' 
 query : string 
}) => any;

 } // namespace functions user What is the minimum lot size, in square feet, for a single family home in the first listed residential district of the town of Ansonia, Connecticut? assistant ```typescript
functions.search_documents({
"town_name": "ansonia",
"query": "zoning district list"
})``` user ARTICLE II - ESTALISHMENT OF DISTRICTS AND ZONING
ADMINISTRATION
Section 205 Zoning Districts Established
For the purposes of these regulations, the City of Ansonia is hereby divided into the following
classes of districts:
Section 210 Zoning Map
The boundaries of the districts specified in Section 205 are hereby established as shown on a
map entitled "Official Zoning Map, City of Ansonia", dated February 1990, including any
special maps and boundary descriptions supplementary thereto, adopted and declared to be a part
of this Ordinance. In addition, the boundaries of the Flood Plain District are as delineated in
Section 220 - Flood Plain District.
Section 215 - Determination of District Boundaries
In determining the boundaries of districts shown on the Official Zoning Map, the following rules
shall apply
215.1 Unless otherwise shown, the district boundaries shall be construed to coincide with
the center lines of streets, alleys, parkways, waterways and main tracks of railroads. -
215.2
Where such boundaries are indicated as approximate following the property lines of
parks or other publicly owned lands, such lines shall be construed to be such
17
CELL (1, 1): 
District
CELL (1, 2): 
Map Code
CELL (2, 1): 
AAA Residence District
CELL (2, 2): 
AAA
CELL (3, 1): 
AA Residence District
CELL (3, 2): 
AA
CELL (4, 1): 
A Residence District
CELL (4, 2): 
A
CELL (5, 1): 
B Residence Di

In [17]:
t.variables()

{'llm': <guidance.llms._openai.OpenAI at 0x2a626d6f0>,
 'question': 'Which districts permit single family homes in the town of Ashford, CT?',
 'functions': [{'name': 'search_documents',
   'description': 'Searches the document store for the contents of a document matching the query. Use this tool to learn more about a topic.',
   'parameters': {'type': 'object',
    'properties': {'town_name': {'type': 'string',
      'description': "The name of the town to search in. Must be in all lowercase and have spaces converted to dashes, e.g. 'Beacon Falls' becomes 'beacon-falls'."},
     'query': {'type': 'string',
      'description': "A keyword search query, e.g. 'AAA Residence District, minimum lot size, single family home'"}},
    'required': ['town_name', 'query']}}],
 'search_documents': <function __main__.search_documents(town_name: str, query: str)>,
 '@raw_prefix': '\n{{!--GMARKER_START_system$&#123;&#123;#system~&#125;&#125;\nYou are ZoningGPT, a powerful assistant who is an expert i

In [18]:
rich.print(t.log)

2023-07-06 23:01:35   [start]         system: {'positional_args': [], 'named_args': {}, '@prefix': '\n'}
2023-07-06 23:01:35   [start]      functions: {'positional_args': [], 'named_args': {}, '@prefix': 
'\n<|im_start|>system\nYou are ZoningGPT, a powerful assistant who is an expert in reviewing portions of Zoning 
documents and answering questions about them.\n\nPay close attention to your answers in the past and be sure to 
match their schema. You will be evaluated on the quality and correctness of your answers.\n\nYou must answer by 
using one of the functions at your disposal, or by returning a JSON formatted final answer. If you cannot answer 
the question, return the string "FAILURE".\n\nYou may use the functions as many times as you like.\n\n'}
2023-07-06 23:01:35     [end]      functions: {'new_prefix': ''}
2023-07-06 23:01:35   [start]      functions: {'positional_args': [], 'named_args': {}, '@prefix': 
'\n<|im_start|>system\nYou are ZoningGPT, a powerful assistant who is an expert in reviewing portions of Zoning 
documents and answering questions about them.\n\nPay close attention to your answers in the past and be sure to 
match their schema. You will be evaluated on the quality and correctness of your answers.\n\nYou must answer by 
using one of the functions at your disposal, or by returning a JSON formatted final answer. If you cannot answer 
the question, return the string "FAILURE".\n\nYou may use the functions as many times as you like.\n\n\n# 
Tools\n\n'}
2023-07-06 23:01:35     [end]      functions: {'new_prefix': ''}
2023-07-06 23:01:35   [start]            len: {'positional_args': [[{'name': 'search_documents', 'description': 
'Searches the document store for the contents of a document matching the query. Use this tool to learn more about a
topic.', 'parameters': {'type': 'object', 'properties': {'town_name': {'type': 'string', 'description': "The name 
of the town to search in. Must be in all lowercase and have spaces converted to dashes, e.g. 'Beacon Falls' becomes
'beacon-falls'."}, 'query': {'type': 'string', 'description': "A keyword search query, e.g. 'AAA Residence 
District, minimum lot size, single family home'"}}, 'required': ['town_name', 'query']}}]], 'named_args': {}, 
'@prefix': '\n<|im_start|>system\nYou are ZoningGPT, a powerful assistant who is an expert in reviewing portions of
Zoning documents and answering questions about them.\n\nPay close attention to your answers in the past and be sure
to match their schema. You will be evaluated on the quality and correctness of your answers.\n\nYou must answer by 
using one of the functions at your disposal, or by returning a JSON formatted final answer. If you cannot answer 
the question, return the string "FAILURE".\n\nYou may use the functions as many times as you like.\n\n\n# 
Tools\n\n'}
2023-07-06 23:01:35     [end]            len: {'new_prefix': ''}
2023-07-06 23:01:35   [start]BINARY_OPERATOR_>: {'positional_args': [1, 0], 'named_args': {}, '@prefix': 
'\n<|im_start|>system\nYou are ZoningGPT, a powerful assistant who is an expert in reviewing portions of Zoning 
documents and answering questions about them.\n\nPay close attention to your answers in the past and be sure to 
match their schema. You will be evaluated on the quality and correctness of your answers.\n\nYou must answer by 
using one of the functions at your disposal, or by returning a JSON formatted final answer. If you cannot answer 
the question, return the string "FAILURE".\n\nYou may use the functions as many times as you like.\n\n\n# 
Tools\n\n'}
2023-07-06 23:01:35     [end]BINARY_OPERATOR_>: {'new_prefix': ''}
2023-07-06 23:01:35   [start]             if: {'positional_args': [True], 'named_args': {}, '@prefix': 
'\n<|im_start|>system\nYou are ZoningGPT, a powerful assistant who is an expert in reviewing portions of Zoning 
documents and answering questions about them.\n\nPay close attention to your answers in the past and be sure to 
match their schema. You will be eva